In [1]:
import numpy as np
import pandas as pd

In [2]:
matchDF = pd.read_csv("./Datasets/matches_2008-2024.csv")
deliveryDF = pd.read_csv("./Datasets/deliveries_2008-2024.csv")

In [3]:
deliveryDF.columns = deliveryDF.columns.str.strip()
matchDF.columns = matchDF.columns.str.strip()

print(deliveryDF.columns.to_list())
print()
print(matchDF.columns.to_list())

['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball', 'batter', 'bowler', 'non_striker', 'batsman_runs', 'extra_runs', 'total_runs', 'extras_type', 'is_wicket', 'player_dismissed', 'dismissal_kind', 'fielder']

['id', 'season', 'city', 'date', 'match_type', 'player_of_match', 'venue', 'team1', 'team2', 'toss_winner', 'toss_decision', 'winner', 'result', 'result_margin', 'target_runs', 'target_overs', 'super_over', 'method', 'umpire1', 'umpire2']


In [4]:
# Merging delivery and match dataframes based on match_id
merged_df = pd.merge(
    deliveryDF,
    matchDF[['id', 'date', 'venue']],
    left_on='match_id',
    right_on='id',
    how='left'
)

In [5]:
merged_df.head()

,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,extra_runs,total_runs,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder,id,date,venue
0,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,1,SC Ganguly,P Kumar,BB McCullum,0,1,1,legbyes,0,NA,NA,NaN,335982,2008-04-18,M Chinnaswamy Stadium
1,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,2,BB McCullum,P Kumar,SC Ganguly,0,0,0,,0,NA,NA,NaN,335982,2008-04-18,M Chinnaswamy Stadium
2,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,3,BB McCullum,P Kumar,SC Ganguly,0,1,1,wides,0,NA,NA,NaN,335982,2008-04-18,M Chinnaswamy Stadium
3,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,4,BB McCullum,P Kumar,SC Ganguly,0,0,0,,0,NA,NA,NaN,335982,2008-04-18,M Chinnaswamy Stadium
4,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,5,BB McCullum,P Kumar,SC Ganguly,0,0,0,,0,NA,NA,NaN,335982,2008-04-18,M Chinnaswamy Stadium


In [6]:
# Selecting and renaming relevant columns
newdataset = merged_df[['match_id', 'date', 'venue', 'batting_team', 'bowling_team',
                        'batter', 'bowler', 'total_runs', 'player_dismissed', 'over', 'ball']].copy()


newdataset.rename(columns={
    'match_id': 'mid',
    'batting_team': 'bat_team',
    'bowling_team': 'bowl_team',
    'batter': 'batsman',
    'total_runs': 'runs'
}, inplace=True)


newdataset['mid'] = newdataset['mid'].astype('int64') - 335981

newdataset.head(10)

,mid,date,venue,bat_team,bowl_team,batsman,bowler,runs,player_dismissed,over,ball
0,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,P Kumar,1,NA,0,1
1,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,0,NA,0,2
2,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,1,NA,0,3
3,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,0,NA,0,4
4,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,0,NA,0,5
5,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,0,NA,0,6
6,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,1,NA,0,7
7,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,Z Khan,0,NA,1,1
8,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,Z Khan,4,NA,1,2
9,1,2008-04-18,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,Z Khan,4,NA,1,3


In [25]:
# 'wickets' by counting non-null values in 'player_dismissed' within each 'mid' group
newdataset['wickets'] = newdataset.groupby(
    'mid')['player_dismissed'].transform(lambda x: x.notnull().cumsum()
                                         )

# overs as a floating point (e.g., 0.1, 0.2...) based on 'over' and 'ball'
newdataset['overs'] = newdataset['over'] + (newdataset['ball'] - 1) * 0.1

# cumulative runs and total score for each match
newdataset['total'] = newdataset.groupby('mid')['runs'].cumsum()


# last 5 overs runs and wickets using a rolling window of 30 deliveries
newdataset['runs_last_5'] = newdataset.groupby('mid')['runs'].transform(
    lambda x: x.rolling(30, min_periods=1).sum()
)
# For wickets, we take the last cumulative count within each rolling window
newdataset['wickets_last_5'] = newdataset.groupby('mid')['wickets'].transform(
    lambda x: x.rolling(30, min_periods=1).apply(
        lambda y: y.iloc[-1], raw=False)
)


# cumulative runs for striker and non-striker
newdataset['striker'] = newdataset.groupby('mid')['runs'].cumsum()
newdataset['non_striker'] = newdataset.groupby(
    'mid')['runs'].cumsum().shift(1).fillna(0).astype(int)

# Selecting final columns for the dataset
final_dataset = newdataset[
    ['mid', 'date', 'venue', 'bat_team', 'bowl_team', 'batsman', 'bowler', 'runs',
        'wickets', 'overs', 'runs_last_5', 'wickets_last_5', 'striker', 'non_striker', 'total']
]

# Displaying the head of the final dataset
final_dataset.head()

In [26]:
final_dataset['total'].to_list()

In [27]:
final_dataset.to_csv('./Datasets/scorePridiction.csv', index=False)